**Data Skew**
Data skew means data is unevenly distributed, causing some tasks to do much more work than others.

Imagine 4 people carrying boxes:

Person 1 → 1 box<br>
Person 2 → 1 box<br>
Person 3 → 1 box<br>
Person 4 → 100 boxes 😓<br>

Everyone must wait for Person 4 to finish.<br>

👉 That imbalance is data skew.<br>

Data Skew in Spark (Very Simple)

Spark splits data into partitions.<br>
Each partition is processed by one task.<br>

**Ideal case**

Partition 1 → 1M rows<br>
Partition 2 → 1M rows<br>
Partition 3 → 1M rows<br>
Partition 4 → 1M rows<br>
All tasks finish together ✅<br>

**Skewed case**
Partition 1 → 50 rows<br>
Partition 2 → 70 rows<br>
Partition 3 → 80 rows<br>
Partition 4 → 10M rows 😱<br>

3 tasks finish quickly<br>
1 task runs forever<br>

👉 Job is slow → data skew<br>

Why Data Skew Happens<br>
1️⃣ Skewed values in a column<br>
Example:<br>
country = 'IN' → 90% of rows<br>
country = others → 10%<br>
Partitioning or grouping by country causes skew.<br>

1️⃣ JOIN Skew (Most Common)
❌ Problem

Joining on a skewed key<br>
Example:<br>

SELECT *<br>
FROM orders o<br>
JOIN customers c<br>
ON o.country = c.country;<br>
Data:<br>

country = 'IN' → 90% of rows<br>

⚠️ Why Skew Happens<br>
All IN rows go to one shuffle partition<br>
One task becomes huge<br>
Job waits for that task<br>

✅ Solutions<br>
✅ Best: Broadcast JOIN (if one table is small)<br>

SELECT /*+ BROADCAST(customers) */ *<Br>
FROM orders o<br>
JOIN customers c<br>
ON o.country = c.country;<br>

Rows with the same join key must be processed by the same task.<br>
Otherwise Spark could not match orders.IN with customers.IN.<br>
Before the join, Spark performs a shuffle on both tables.<br>

Each row is sent to a partition based on:<br>
partition_id = hash(join_key) % num_shuffle_partitions<br>
here join_key = country<br>

Why All 'IN' Rows Go Together<br>
country = 'IN' → 90% of rows<br>
Important fact:<br>
hash("IN") is always the same<br>
Hashing is deterministic<br>
So for every row:
hash("IN") % N  → same partition number<br>
👉 Spark must do this to make the join correct.<br>

**Visualizing the Shuffle**

Imagine spark.sql.shuffle.partitions = 4<br>
| Country | hash(country) % 4 | Partition       |
| ------- | ----------------- | --------------- |
| IN      | 2                 | **Partition 2** |
| US      | 0                 | Partition 0     |
| UK      | 1                 | Partition 1     |
| FR      | 3                 | Partition 3     |

Now because 90% = IN:

Partition 2 → 90% of data 😱<br>
Other partitions → tiny<br>

Key Rule (Very Important)<br>

Spark does NOT distribute rows evenly<br>

Spark groups rows by join key value, not by row count.<br>

✅ Solutions<br>
✅ Best: Broadcast JOIN (if one table is small)<br>
What changes with BROADCAST JOIN<br>
Key idea:<br>

Small table is sent to every executor — no shuffle on join key<br>
Physical plan:<br>
orders  ──▶ scan (NO shuffle)<br>
customers ──▶ broadcast to all executors<br>


SELECT /*+ BROADCAST(customers) */ *<br>
FROM orders o<br>
JOIN customers c<br>
ON o.country = c.country;<br>

✅ Alternative: Salting
Salting is used when both tables are large, so broadcast is impossible<br>
-- Add salt on both sides<br>
SELECT *<br>
FROM orders o<br>
JOIN customers c<br>
ON o.country = c.country<br>
AND o.salt = c.salt;<br>

-----

2️⃣ GROUP BY Skew<br>
❌ Problem<br>

Grouping on a dominant value<br>
🧪 Example<br>

SELECT country, COUNT(*)<br>
FROM orders<br>
GROUP BY country;<br>

⚠️ Why Skew Happens

One group (IN) holds most rows

One reducer processes huge data

In a GROUP BY, Spark repartitions the data by the grouping key, and each group key is assigned to exactly one <br>
shuffle partition. If one key (like IN) has most rows, the task processing that partition becomes skewed.<br>

For correctness:<br>
All rows of the same country must be processed by the same task.<br>
Otherwise Spark could not compute COUNT(*) correctly.<br>
So Spark must collect all IN rows together.<br>

✅ Salting + Re-aggregation

```sql
-- Step 1: Salt
WITH salted AS (
  SELECT
    country,
    FLOOR(rand() * 10) AS salt
  FROM orders
)

-- Step 2: Aggregation
SELECT
  country,
  COUNT(*)
FROM salted
GROUP BY country;

✅ AQE (runtime fix)<br>
SET spark.sql.adaptive.enabled = true;<br>
SET spark.sql.adaptive.skewJoin.enabled = true;<br>
-----
3️⃣ Partitioning Skew (Delta / Hive)
❌ Problem

Partitioning by a skewed column

🧪 Example

```sql
CREATE TABLE sales (
  order_id INT,
  country STRING
)'''
USING DELTA
PARTITIONED BY (country);
⚠️ Why Skew Happens

country = IN folder contains most files

Queries on that partition are slow

Small countries → tiny partitions

✅ Solutions

✅ Avoid partitioning
CLUSTER BY (customer_id)

✅ Use Liquid Clustering
CREATE TABLE sales
USING DELTA
CLUSTER BY (customer_id);


4️⃣ ORDER BY Skew (Single Reducer Problem)
❌ Problem

Using ORDER BY

🧪 Example
SELECT *
FROM orders
ORDER BY order_date;

⚠️ Why Skew Happens

ORDER BY forces single reducer

One task processes all data

✅ Solutions
✅ Use SORT BY
SELECT *
FROM orders
SORT BY order_date;

✅ Or DISTRIBUTE BY + SORT BY
SELECT *
FROM orders
DISTRIBUTE BY order_date
SORT BY order_date;

🎯 Interview Note

ORDER BY = single partition → guaranteed skew.

5️⃣ Window Function Skew
❌ Problem

Window partition on skewed key

🧪 Example
SELECT *,
       ROW_NUMBER() OVER (PARTITION BY country ORDER BY order_date)
FROM orders;

⚠️ Why Skew Happens

All IN rows processed by one task

✅ Solutions
✅ Reduce partition size
PARTITION BY country, year(order_date)

✅ Filter early
WHERE order_date >= current_date() - 30



####1. Handling Data Skew & Query Performance (Optimize & Z-Order)
Scenario: The analytics team reports that queries filtering silver_shipments by source_city and shipment_date are becoming slow as data volume grows.

Task: Run the OPTIMIZE command with ZORDER on the silver_shipments table to co-locate related data in the same files.

Outcome:
Why did we choose source_city and shipment_date for Z-Ordering instead of shipment_id? Think about high cardinality vs. query filtering

In [0]:
%sql
USE prodcatalog1.logistics1

In [0]:
%sql
SELECT * FROM silver_shipments
limit 10

Since queries are slow when filtering on source_city and shipment_date, we explicitly optimize file layout using Z-Ordering

In [0]:
%sql
DESCRIBE DETAIL silver_shipments

In [0]:
%sql
OPTIMIZE silver_shipments
ZORDER BY (source_city,shipment_date);

-- What this does
-- Rewrites many small Delta files into fewer, larger files
-- Physically co-locates rows with similar source_city and shipment_date
-- Improves data skipping during query execution
-- Reduces I/O and scan time for filter-heavy queries

-- Z-ORDER is most effective on columns frequently used in query filters and with medium to low cardinality

-- source_city
-- Frequently used in WHERE clauses
-- Limited set of values (cities repeat)
-- High data locality benefit
-- Enables skipping entire files when city doesn’t match

-- Why NOT shipment_id
--Very high cardinality AND Rarely used in filters

In [0]:
%sql
DESCRIBE DETAIL silver_shipments

#### 2. Speeding up Regional Queries (Partition Pruning)
Scenario: The dashboard team reports that queries filtering for orgin_hub_city with "New York" shipments from the gold_core_curated_tbl table are scanning the entire dataset (Terabytes of data), even though New York is only 5% of the data. This is racking up compute costs.

Task: Re-create the gold_core_curated_tbl table partitioned by orgin_hub_city. Run a query filtering for one city to demonstrate "Partition Pruning" (where Spark skips files that don't match the filter).

Outcome: Verify the partition filtering is applied or not, by performing explain plan, check for the PartitionFilters in the output.

In [0]:
%sql
SELECT * FROM core_curated_tbl
LIMIT 5

In [0]:
%sql
CREATE OR REPLACE TABLE Gold_core_curated_tbl
USING DELTA
PARTITIONED BY (origin_hub_city)
AS
SELECT * FROM core_curated_tbl;


In [0]:
%sql
EXPLAIN select * from Gold_core_curated_tbl
WHERE origin_hub_city = 'London'
LIMIT 5

In [0]:
%sql
DESC DETAIL Gold_core_curated_tbl

In [0]:
%sql
SHOW PARTITIONS Gold_core_curated_tbl

#### 3. Storage Cost Savings (Vacuum)
Scenario: Your Project pipeline runs every hour, creating many small files and obsolete versions of data. Your storage costs are rising. You need to clean up files that are no longer needed for time travel.

Task: Execute a Vacuum command to remove data files older than the retention threshold.

Outcome: Perform the describe history and find whether vacuum is completed.

In [0]:
%sql
DESC HISTORY core_curated_tbl

In [0]:
%sql
VACUUM core_curated_tbl RETAIN 168 hours

In [0]:
%sql
DESC HISTORY core_curated_tbl

####4. Modern Data Layout (Liquid Clustering)
Scenario: You are redesigning the silver_shipments table. You want to avoid the "small files" problem and need a flexible layout that adapts to changing query patterns automatically without rewriting the table.

Task: Re-create the silver_shipments table using Liquid Clustering on the shipment_id column.

Outcome: Liquid Clustering over traditional partitioning when the cardinality of shipment_id is very high.

In [0]:
%sql
CREATE OR REPLACE TABLE silver_shipments_liquid
USING DELTA
CLUSTER BY (shipment_id)
AS
SELECT * FROM silver_shipments

In [0]:
%sql
DESC HISTORY silver_shipments_liquid

#### 5. Cost Efficient Environment Cloning (Shallow Clone)
Scenario: The QA team needs to test an update on the gold_core_curated_tbl table. The table is 5TB in size. You cannot afford to duplicate the storage cost just for a test and the update should not affect the copied table.

Task: Create a Shallow Clone of the gold table for the QA team.

Outcome: If we delete records from the source table (gold_core_curated_tbl), will the QA table (gold_core_curated_tbl_qa) be affected? Why or why not?

In [0]:
%sql
SELECT * FROM core_curated_tbl
WHERE shipment_id=5010041

In [0]:
%sql
CREATE OR REPLACE TABLE core_curated_tbl_QA_Clone
SHALLOW CLONE core_curated_tbl

In [0]:
%sql
SELECT * FROM core_curated_tbl_QA_Clone
WHERE shipment_id=5010041

In [0]:
%sql
DELETE FROM core_curated_tbl_QA_Clone
WHERE shipment_id=5010041

In [0]:
%sql
SELECT * FROM core_curated_tbl_QA_Clone
WHERE shipment_id=5010041

In [0]:
%sql
SELECT * FROM core_curated_tbl
WHERE shipment_id=5010041

Each table has its own transaction log, so deletes only change the metadata of the table where the operation is executed.

A shallow clone points to the same data files as the original table, but it has its own transaction log. If a transaction happens on the original table, only the original table’s metadata is updated. No metadata changes are written to the clone, so the clone remains unchanged.

#### 6. Disaster Recovery (Time Travel & Restore)
Scenario: A junior data engineer accidentally ran a logic error that corrupted the gold_core_curated_tbl table 15 minutes ago. You need to revert the table to its previous state immediately.

Task: Use Delta Lake's Restore feature to roll back the table.

Outcome:What is the difference between querying with VERSION AS OF (Time Travel) and running RESTORE?

In [0]:
%sql
DESC HISTORY core_curated_tbl

In [0]:
%sql
SELECT * FROM core_curated_tbl VERSION AS OF 4

In [0]:
%sql
-- Fix: Use a supported timestamp literal for RESTORE
-- Example: RESTORE TABLE ... TO TIMESTAMP AS OF '2026-01-31T10:00:00Z'
-- You can find the correct timestamp from DESC HISTORY output

-- Replace with a valid timestamp from your table history
RESTORE TABLE prodcatalog1.logistics1.core_curated_tbl TO TIMESTAMP AS OF '2026-01-31T10:00:00Z';

In [0]:
%sql
restore table core_curated_tbl to version as of 5

Time Travel (VERSION AS OF) = temporary read of old data<br>
- What happens
- Only your query sees version 10
- Other users still see the latest (broken) data
- As soon as the query finishes → old data is gone from view<br>
Key point<br>
❌ Table is NOT fixed<br>

RESTORE = permanent rollback of the table<br>
What happens
- The table itself is rolled back
- Everyone now sees version 10
- A new version is created showing the restore action

Key point
✅ Table IS fixed